In [1]:
import numpy as np

In [2]:
np.random.seed(42)
X = np.random.randn(1_000_000, 100)
y = np.random.randn(1_000_000) < 0.0

In [3]:
X[y].base is X

False

In [4]:
np.compress(y, X, axis=0).base is X

False

In [5]:
%timeit -n100 -r7 X[y]

162 ms ± 9 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit -n100 -r7 np.compress(y, X, axis=0)

124 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
from numba import njit

In [8]:
@njit(cache=True)
def select(X, y):
    return X[y]

select(X, y)
None

In [9]:
%timeit -n10 -r7 select(X, y)

265 ms ± 6.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
def foo1(X, _feature_idx):
    feature = X[:, _feature_idx]
    thr = np.mean(feature)
    left = X[feature < thr]
    right = X[feature > thr]
    return left, right

def foo(X, _feature_idx):
    feature = X[:, _feature_idx]
    thr = np.mean(feature)
    left_idx = feature < thr
    right_idx = np.logical_not(left_idx)
    left = np.compress(left_idx, X, axis=0)
    right = np.compress(right_idx, X, axis=0)
    return left, right


@njit(parallel=True, cache=True)
def foo_c(X, _feature_idx):
    feature = X[:, _feature_idx]
    thr = np.mean(feature)
    left_idx = feature < thr
    right_idx = np.logical_not(left_idx)
    left = X[left_idx]
    right = X[right_idx]
    return left, right

foo_c(X, 0)
None

In [11]:
%timeit -n10 -r7 foo1(X, 0)

352 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit -n10 -r7 foo(X, 0)

253 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit -n10 -r7 foo_c(X, 0)

530 ms ± 5.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
